In [2]:
#import externally developed packages
import os
import sys
import vcfpy
import re
import pandas as pd

## Import a .vcf file and convert it into a dataframe with a unique identifier for each SNP

In [3]:
##import vcf

#specify path of vcf
vcf_path = '/Users/tristanpwdennis/Software/seqtoolbox/analysis-scripts/vcf/vcf/test.vcf'

#import vcf as reader object
reader = vcfpy.Reader.from_path(vcf_path)

# Build header
header = pd.DataFrame.transpose(pd.DataFrame(['#CHROM', 'POS', 'REF', 'ALT'] + reader.header.samples.names))

# Design matrix in list format
X_list = []  

#convert vcf convert to dataframe - extract GT field, chrom, pos ref and alt data
for record in reader:
    if not record.is_snv():
        continue
    line = [record.CHROM, record.POS, record.REF]
    line += [alt.value for alt in record.ALT]
    line += [call.data.get('GT') or './.' for call in record.calls]
    X_list.append(line)
    df = pd.DataFrame(X_list)

#stick header onto the genotype frame
df=header.append(df) 

#make added row into the df header
new_header = df.iloc[0] #grab the first row for the header
df = df[1:] #take the data less the header row
df.columns = new_header #set the header row as the df header

In [4]:
##vcf to colony input
#make unique genotype id from chrom, pos ref and alt
id=df["#CHROM"].map(str) + df["POS"].map(str) + df["REF"].map(str) + df["ALT"].map(str)
#add new header
df.insert(0, 'id', id)
#drop chrom pos ref and alt fields
gt=df.drop(['#CHROM', 'POS', 'REF', 'ALT'], axis=1)

#specify mapping (from genotype values to codes)
di = {
    "./.": 0,
    "0/0": 1,
    "0/1": 2,
    "1/0": 3,
    "1/1": 4,
}  

#rename genotypes to be codes following mapping above
gt=gt.replace(di)

#transpose into individuals as columns and genotypes as rows
gt=gt.transpose()

#header twattery again
new_header = gt.iloc[0] #grab the first row for the header
gt = gt[1:] #take the data less the header row
gt.columns = new_header #set the header row as the df header

In [5]:
gt.head()

id,OVOC_OO_0003271374CT,OVOC_OO_0003271375GA,OVOC_OO_0003271571AC,OVOC_OO_0003271612TC,OVOC_OO_0003271629GT,OVOC_OO_0003271766AT,OVOC_OO_0003271785TC,OVOC_OO_0003271846AG,OVOC_OO_0003271958AG,OVOC_OO_0003272094AG,...,OVOC_OO_00002410258TG,OVOC_OO_00002410264GA,OVOC_OO_00002410268TC,OVOC_OO_00002410299TA,OVOC_OO_00002411392AC,OVOC_OO_00002411432TA,OVOC_OO_00002411443TC,OVOC_OO_00002411561CT,OVOC_OO_00002411823AT,OVOC_OO_00002412197GT
0,,,,,,,,,,,,,,,,,,,,,
ind0,1,1,1,1,1,1,1,1,1,1,...,1,1,1,2,1,2,2,1,2,2
ind1,2,1,1,1,1,1,1,1,1,1,...,1,1,1,2,1,2,2,2,2,2
ind2,1,2,1,1,1,2,1,1,1,1,...,2,2,1,2,1,1,2,2,4,2
ind3,1,1,1,2,1,1,1,2,2,1,...,2,1,2,2,1,4,2,2,2,2
ind4,2,2,1,1,1,1,1,1,2,1,...,1,1,4,2,2,1,1,4,4,4


In [8]:
df=pd.read_table('test.geno', sep='\t',)

In [9]:
df

,OVOC_OO_000327,1374,NN,CT,CC,CC.1,CT.1,CT.2,CT.3,CC.2,...,NN.10,CC.18,CC.19,CC.20,CC.21,CT.14,CT.15,CT.16,NN.11,Unnamed: 53
0,OVOC_OO_000327,1571,NN,AA,AA,AA,AA,AA,AC,AA,...,AA,AA,AA,AA,AA,AC,AA,AA,NN,NaN
1,OVOC_OO_000327,1612,NN,TT,TT,TC,TT,TT,TT,TT,...,NN,TT,TC,TT,NN,TT,TT,TC,NN,NaN
2,OVOC_OO_000327,1629,NN,GG,GG,GG,GG,GG,GG,GG,...,NN,GG,GG,GT,GG,GG,GG,GG,NN,NaN
3,OVOC_OO_000327,1766,NN,AA,AT,AA,AA,AA,AA,AT,...,NN,AT,AA,NN,AA,AA,AA,AA,AA,NaN
4,OVOC_OO_000327,1785,NN,TT,TT,TT,TT,TT,TC,TT,...,TT,TC,TC,NN,TT,TT,TC,TT,NN,NaN
5,OVOC_OO_000327,1846,NN,AA,AA,AG,AA,AG,AA,AG,...,AA,AA,AG,AG,AA,AA,AA,AA,NN,NaN
6,OVOC_OO_000327,1958,NN,AA,AA,AG,AG,AG,AG,AG,...,AA,AA,AG,AG,AG,AG,AA,AA,NN,NaN
7,OVOC_OO_000327,2094,NN,NN,AA,AA,AA,AA,AA,AA,...,NN,AA,AA,AA,AA,AA,AA,AA,NN,NaN
8,OVOC_OO_000327,2182,NN,AA,AC,AA,AA,AA,AA,AA,...,NN,AA,AA,AC,AA,AA,NN,AA,NN,NaN


In [12]:
df.iloc[ : , 0 ]

0    OVOC_OO_000327
1    OVOC_OO_000327
2    OVOC_OO_000327
3    OVOC_OO_000327
4    OVOC_OO_000327
5    OVOC_OO_000327
6    OVOC_OO_000327
7    OVOC_OO_000327
8    OVOC_OO_000327
Name: OVOC_OO_000327, dtype: object